<a href="https://colab.research.google.com/github/vishnoiprem/Chargebee-etl/blob/master/kafka_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Easiest way to download kaggle data in Google Colab
Posted in General 3 years ago
698

Please follow the steps below to download and use kaggle data within Google Colab:

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

3. Go to your Google Colab project file and run the following commands:

1) ! pip install -q kaggle

2) from google.colab import files

files.upload()

Choose the kaggle.json file that you downloaded
3) ! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

Make directory named kaggle and copy kaggle.json file there.
4) ! chmod 600 ~/.kaggle/kaggle.json

Change the permissions of the file.
5) ! kaggle datasets list
- That's all ! You can check if everything's okay by running this command.

Download Data
! kaggle competitions download -c 'name-of-competition'

Use unzip command to unzip the data:

For example,

Create a directory named train,

! mkdir train

unzip train data there,

! unzip train.zip -d train

https://www.kaggle.com/general/74235

In [23]:
!pip install -q kaggle

In [24]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vishnoiprem","key":"0de5a6f99c0213829a88f6707c801532"}'}

# New section

3) ! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [26]:
! mkdir ~/.kaggle


In [27]:

!cp kaggle.json ~/.kaggle/

In [28]:
!chmod 600 ~/.kaggle/kaggle.json

In [29]:
# !kaggle competitions download -c 'wordsforthewise'
!kaggle datasets download -d wordsforthewise/lending-club

 99% 1.25G/1.26G [00:10<00:00, 160MB/s]
100% 1.26G/1.26G [00:11<00:00, 123MB/s]


In [30]:
!ls -lrt

total 1324728
drwxr-xr-x 1 root root       4096 Jun  1 13:50 sample_data
-rw-r--r-- 1 root root         67 Jun  4 11:25 kaggle.json
-rw-r--r-- 1 root root 1356507910 Jun  4 11:25 lending-club.zip


In [31]:
!unzip lending-club.zip

Archive:  lending-club.zip
  inflating: accepted_2007_to_2018Q4.csv.gz  
  inflating: accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv  
  inflating: rejected_2007_to_2018Q4.csv.gz  
  inflating: rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv  


In [32]:
!ls

accepted_2007_to_2018q4.csv	lending-club.zip		sample_data
accepted_2007_to_2018Q4.csv.gz	rejected_2007_to_2018q4.csv
kaggle.json			rejected_2007_to_2018Q4.csv.gz


In [33]:
!ls

accepted_2007_to_2018q4.csv	lending-club.zip		sample_data
accepted_2007_to_2018Q4.csv.gz	rejected_2007_to_2018q4.csv
kaggle.json			rejected_2007_to_2018Q4.csv.gz


In [34]:
!pip install kafka-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 6.3 MB/s 


In [68]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [35]:
#https://nb.recohut.com/spark/pyspark/kafka/movie/2021/06/25/kafka-spark-streaming-colab.html

In [36]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from sklearn.model_selection import train_test_split


In [37]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

In [38]:
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [77]:
!ps -ef | grep kafka


root         768       1  0 11:26 ?        00:00:05 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.1.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.1.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.1.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.1.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [40]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic susy-train
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic susy-test

Created topic susy-train.
Created topic susy-test.


In [41]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic susy-train
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic susy-test


Topic: susy-train	TopicId: f1H2DVajTkush9gzeJ3jFQ	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-train	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: susy-test	TopicId: x47LhTtrRYqQ1BrOYBP5gw	PartitionCount: 2	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: susy-test	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [42]:
susy_iterator = pd.read_csv('rejected_2007_to_2018Q4.csv.gz', chunksize=5)
susy_df = next(susy_iterator)
susy_df.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [43]:
len(susy_df), len(susy_df.columns)

(5, 9)

In [44]:
# Number of datapoints belonging to each class (0: background noise, 1: signal)
d = susy_df.to_json(orient='records')
d

'[{"Amount Requested":1000.0,"Application Date":"2007-05-26","Loan Title":"Wedding Covered but No Honeymoon","Risk_Score":693.0,"Debt-To-Income Ratio":"10%","Zip Code":"481xx","State":"NM","Employment Length":"4 years","Policy Code":0.0},{"Amount Requested":1000.0,"Application Date":"2007-05-26","Loan Title":"Consolidating Debt","Risk_Score":703.0,"Debt-To-Income Ratio":"10%","Zip Code":"010xx","State":"MA","Employment Length":"< 1 year","Policy Code":0.0},{"Amount Requested":11000.0,"Application Date":"2007-05-27","Loan Title":"Want to consolidate my debt","Risk_Score":715.0,"Debt-To-Income Ratio":"10%","Zip Code":"212xx","State":"MD","Employment Length":"1 year","Policy Code":0.0},{"Amount Requested":6000.0,"Application Date":"2007-05-27","Loan Title":"waksman","Risk_Score":698.0,"Debt-To-Income Ratio":"38.64%","Zip Code":"017xx","State":"MA","Employment Length":"< 1 year","Policy Code":0.0},{"Amount Requested":1500.0,"Application Date":"2007-05-27","Loan Title":"mdrigo","Risk_Score"

In [45]:
# def error_callback(exc):
#     raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

# def write_to_kafka(topic_name, items):
#   count=0
#   producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
#   for message, key in items:
#     print(message, key)
#     producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
#     count+=1
#   producer.flush()
#   print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [46]:
# write_to_kafka("susy-train",d[0] )


ValueError: ignored

In [ ]:
# def custom_fun(row):
#   producer.send('susy-train',str(row.asDict()))
#   producer.flush()


In [ ]:
# susy_df.foreach(custom_fun)


In [ ]:
from kafka import KafkaProducer


def send_to_kafka(rows):
    producer = KafkaProducer(bootstrap_servers = util.get_broker_metadata())
    for row in rows:
        producer.send('susy-train',str(row.asDict()))  
        producer.flush()

susy_df.foreachPartition(send_to_kafka)

In [ ]:
json(d)

In [47]:
from kafka import KafkaAdminClient, KafkaConsumer, KafkaProducer
from kafka.admin import NewTopic
import json
from json import loads
from csv import DictReader

def kafka_producer_csv():
  # Required setting for Kafka Producer
  bootstrap_servers = ['localhost:9092']
  topicname = 'test.topic.raw'
  producer = KafkaProducer(bootstrap_servers = bootstrap_servers)
  producer = KafkaProducer()
  # iterate over each line as a ordered dictionary and print only few column by column name
  with open('rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv','r') as read_obj:
      csv_dict_reader = DictReader(read_obj)
      for row in csv_dict_reader:
          ack = producer.send(topicname, json.dumps(row).encode('utf-8'))
          metadata = ack.get()
          # print(metadata.topic, metadata.partition)

threading.Thread(target=kafka_producer_csv).start()

In [48]:
!ls -lrt rejected_2007_to_2018q4.csv/

total 1740516
-rw-r--r-- 1 root root 1782281620 Dec 17  2019 rejected_2007_to_2018Q4.csv


In [ ]:
# from kafka import KafkaAdminClient, KafkaConsumer, KafkaProducer
# import sys
# import json
# from json import loads

# ### Setting up the Python consumer
# bootstrap_servers = ['localhost:9092']
# topicName = 'test.topic.raw'
# consumer = KafkaConsumer (topicName, group_id = 'my_group_id1',bootstrap_servers = bootstrap_servers,
#     auto_offset_reset = 'earliest',value_deserializer=lambda x: loads(x.decode('utf-8')))  ## You can also set it as latest
    
# ### Reading the message from consumer
# try:
#     for message in consumer:
#         print (message.value)
# except KeyboardInterrupt:
#     sys.exit()
    

In [49]:
!pip install apache-flink



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
# !./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
# !./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
# !echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
# !sleep 10

!ls -lrt

total 2262880
-rw-r--r--  1 root root  392582231 Dec 17  2019 accepted_2007_to_2018Q4.csv.gz
-rw-r--r--  1 root root  255470782 Dec 17  2019 rejected_2007_to_2018Q4.csv.gz
drwxr-xr-x 13 1000 1000       4096 Jun  6  2020 spark-3.0.0-bin-hadoop3.2
-rw-r--r--  1 root root  224453229 Jun  6  2020 spark-3.0.0-bin-hadoop3.2.tgz
drwxr-xr-x  1 root root       4096 Jun  1 13:50 sample_data
-rw-r--r--  1 root root         67 Jun  4 11:25 kaggle.json
-rw-r--r--  1 root root 1356507910 Jun  4 11:25 lending-club.zip
drwxr-xr-x  2 root root       4096 Jun  4 11:25 accepted_2007_to_2018q4.csv
drwxr-xr-x  2 root root       4096 Jun  4 11:25 rejected_2007_to_2018q4.csv
-rw-r--r--  1 root root   88130011 Jun  4 11:26 kafka_2.13-3.1.0.tgz
drwxr-xr-x  8 root root       4096 Jun  4 11:26 kafka_2.13-3.1.0


In [94]:
from pyflink.common import Configuration
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.expressions import col


In [95]:
configuration = Configuration()
settings = EnvironmentSettings.new_instance().in_streaming_mode().with_configuration(configuration).build()
t_env = TableEnvironment.create(settings)
table_config = t_env.get_config()
table_config.set("table.exec.mini-batch.enabled", "true")
table_config.set("table.exec.mini-batch.allow-latency", "5 s")
table_config.set("table.exec.mini-batch.size", "5000")




# New section

In [ ]:
#https://nightlies.apache.org/flink/flink-docs-master/docs/dev/python/table/intro_to_table_api/

In [ ]:
# # instantiate table environment
# configuration = Configuration()
# configuration.set("table.exec.mini-batch.enabled", "true")
# configuration.set("table.exec.mini-batch.allow-latency", "5 s")
# configuration.set("table.exec.mini-batch.size", "5000")
# settings = EnvironmentSettings.new_instance().in_streaming_mode().with_configuration(configuration).build()
# t_env = TableEnvironment.create(settings)
# # access flink configuration after table environment instantiation
# table_config = t_env.get_config()
# # set low-level key-value options
# table_config.set("table.exec.mini-batch.enabled", "true")
# table_config.set("table.exec.mini-batch.allow-latency", "5 s")
# table_config.set("table.exec.mini-batch.size", "5000")

In [103]:



# t_env.execute_sql("""
# CREATE CATALOG MyCatalog WITH (
#     'type' = 'hive',
#     'default-database' = 'my_database',
#     'hive-conf-dir' = '/etc/hive/conf'
#   );
# """)

from pyflink.table.catalog import HiveCatalog



In [105]:
# catalog = HiveCatalog("myhive", None, "/etc/hive/conf")

# # Register the catalog
# t_env.register_catalog("myhive", catalog)

# # Create a catalog database
# t_env.execute_sql("CREATE DATABASE mydb WITH (...)")

# # Create a catalog table
# t_env.execute_sql("CREATE TABLE mytable (name STRING, age INT) WITH (...)")

# # should return the tables in current catalog and database.
# t_env.list_tables()



TypeError: ignored

In [97]:
# 2. create source Table
t_env.execute_sql("""
    CREATE TABLE if not exists rejected_2007_to_2018q4 (
        amount_requested DOUBLE,
        application_date STRING,
        loan_title STRING,
        risk_score DOUBLE,   
        debt_to_income_ratio STRING,
        zip_code STRING,
        state STRING,
        employment_length STRING,            
        policy_code BIGINT           
    ) WITH (
  'connector' = 'kafka',
  'topic' = 'test.topic.raw',
  'properties.bootstrap.servers' = 'localhost:9092',
  'properties.group.id' = 'my_group_id1',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
    )
""")

In [98]:



t_env.execute_sql("""
    CREATE TABLE if not exists dwi_risk_rejected (
        amount_requested DOUBLE,
        application_date STRING,
        loan_title STRING,
        risk_score DOUBLE,   
        debt_to_income_ratio STRING,
        zip_code STRING,
        state STRING,
        employment_length STRING,            
        policy_code BIGINT           
    ) WITH (
  'connector' = 'print'
    )
""")

In [99]:
t_env.sql_query("SELECT * FROM rejected_2007_to_2018q4")


Py4JJavaError: ignored

In [62]:
t_env.execute_sql("""
    INSERT INTO  dwi_risk_rejected 
    SELECT 
        amount_requested ,
        application_date ,
        loan_title ,
        risk_score ,   
        debt_to_income_ratio ,
        zip_code ,
        state ,
        employment_length ,            
        policy_code   
        FROM rejected_2007_to_2018q4
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



Py4JJavaError: ignored

In [106]:
!pip3 uninstall hsfs -y
!pip3 install hsfs[hive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.9 MB/s 
     |████████████████████████████████| 132 kB 36.3 MB/s 
     |████████████████████████████████| 11.3 MB 41.2 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 68 kB 5.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 8.8 MB 30.6 MB/s 
     |████████████████████████████████| 138 kB 27.3 MB/s 
     |████████████████████████████████| 59 kB 4.1 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 2.0 MB 32.0 MB/s 
     |████████████████████████████████| 4.0 MB 36.2 MB/s 
  Created wheel for hsfs: filename=hsfs-2.5.9-py3-none-any.whl size=152956 sha256=db5c7e25ac7b0dad5377ef

In [107]:
import hsfs

# TODO: replace the values below: [UUID], [project-name], [api-key]
connection = hsfs.connection(host="[UUID].cloud.hopsworks.ai",   # UUID is from Step 1, above
    project="[bank-core]",                                    # project-name is from Step 2, above
    engine="hive",
    api_key_value="[api-key]")                                   # the API key comes from Step 3, above

fs = connection.get_feature_store()

ImportError: ignored